In [14]:
%matplotlib widget

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [15]:
df = pd.read_csv('data/all_timeline.csv')

In [16]:
df.columns

Index(['date', 'countrycode', 'countrylabel', 'showcountrylabelcases',
       'labelpositioncases', 'showcountrylabeldeaths', 'labelpositiondeaths',
       'showcountrylabelrecovered', 'labelpositionrecovered', 'casesoverride',
       'deathsoverride', 'recoveredoverride',
       'columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit',
       'emergencycasesoverride', 'emergencydeathsoverride', 'totalcases',
       'totaldeaths', 'totalrecovered'],
      dtype='object')

In [17]:
df.head(5)

,date,countrycode,countrylabel,showcountrylabelcases,labelpositioncases,showcountrylabeldeaths,labelpositiondeaths,showcountrylabelrecovered,labelpositionrecovered,casesoverride,deathsoverride,recoveredoverride,columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit,emergencycasesoverride,emergencydeathsoverride,totalcases,totaldeaths,totalrecovered
0,1/22/20,AF,Afghanistan,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/22/20,AL,Albania,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,THIS SPREADSHEET IS THE OLD VERSION. DO NOT US...,NaN,NaN,NaN,NaN,NaN
2,1/22/20,DZ,Algeria,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,TO UPDATE UK NUMBER USE THIS NEW SPREADSHEET P...,NaN,NaN,NaN,NaN,NaN
3,1/22/20,AO,Angola,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/22/20,AR,Argentina,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
COUNTRY_CODE = 'BR'

target_country = df[(df['countrycode'] == COUNTRY_CODE) & (df['totalcases'].notna())]
target_country.head()

,date,countrycode,countrylabel,showcountrylabelcases,labelpositioncases,showcountrylabeldeaths,labelpositiondeaths,showcountrylabelrecovered,labelpositionrecovered,casesoverride,deathsoverride,recoveredoverride,columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit,emergencycasesoverride,emergencydeathsoverride,totalcases,totaldeaths,totalrecovered
6284,2/26/20,BR,Brazil,yes,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6463,2/27/20,BR,Brazil,yes,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6642,2/28/20,BR,Brazil,yes,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6821,2/29/20,BR,Brazil,yes,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
7000,3/1/20,BR,Brazil,yes,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


In [19]:
# Remove Unused Columns
del target_country['labelpositioncases']
del target_country['showcountrylabeldeaths']
del target_country['labelpositiondeaths']
del target_country['showcountrylabelrecovered']
del target_country['labelpositionrecovered']
del target_country['showcountrylabelcases']
del target_country['columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit']
del target_country['casesoverride']
del target_country['deathsoverride']
del target_country['recoveredoverride']
del target_country['emergencycasesoverride']
del target_country['emergencydeathsoverride']
del target_country['totaldeaths']
del target_country['totalrecovered']

In [20]:
# Compute Cases Evolution per Day
target_country.loc[:, ('evolution_rate_previous_day')] = 0
evolution_rate_previus_day_column_ix = target_country.columns.get_loc('evolution_rate_previous_day')
totalcases_column_ix = target_country.columns.get_loc('totalcases')

for i in range(1, len(target_country.index)):
    target_country.iloc[i, evolution_rate_previus_day_column_ix] = \
        (target_country.iloc[i, totalcases_column_ix] / target_country.iloc[i-1, totalcases_column_ix])-1

c:\projetos\covid-19\venv\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\projetos\covid-19\venv\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
# Compute New Cases per Day
target_country.loc[:, ('new_cases_per_day')] = 0
new_cases_per_day_column_ix = target_country.columns.get_loc('new_cases_per_day')
totalcases_column_ix = target_country.columns.get_loc('totalcases')

for i in range(1, len(target_country.index)):
    target_country.iloc[i, new_cases_per_day_column_ix] = \
        target_country.iloc[i, totalcases_column_ix] - target_country.iloc[i-1, totalcases_column_ix]

In [29]:
# Add Column With Previous Day new Cases And Previous Day Total Cases
target_country.loc[:, ('previous_day_new_cases')] = 0
target_country.loc[:, ('previous_day_total_cases')] = 0

previous_day_new_cases_column_ix = target_country.columns.get_loc('previous_day_new_cases')
new_cases_per_day_column_ix = target_country.columns.get_loc('new_cases_per_day')
previous_day_total_cases_column_ix = target_country.columns.get_loc('previous_day_total_cases')
total_cases_column_ix = target_country.columns.get_loc('totalcases')

for i in range(1, len(target_country.index)):
    target_country.iloc[i, previous_day_new_cases_column_ix] = target_country.iloc[i-1, new_cases_per_day_column_ix]
    target_country.iloc[i, previous_day_total_cases_column_ix] = target_country.iloc[i-1, total_cases_column_ix]    

In [30]:
# Remove Rows with new_cases_per_day=0
target_country = target_country[target_country['new_cases_per_day'] > 0.0]

In [31]:
target_country.head(50)

,date,countrycode,countrylabel,totalcases,evolution_rate_previous_day,new_cases_per_day,previous_day_new_cases,previous_day_total_cases
6821,2/29/20,BR,Brazil,2.0,1.000000,1.0,0.0,0.0
7537,3/4/20,BR,Brazil,4.0,1.000000,2.0,1.0,2.0
7895,3/6/20,BR,Brazil,13.0,2.250000,9.0,2.0,4.0
8253,3/8/20,BR,Brazil,20.0,0.538462,7.0,9.0,13.0
8432,3/9/20,BR,Brazil,25.0,0.250000,5.0,7.0,20.0
8611,3/10/20,BR,Brazil,31.0,0.240000,6.0,5.0,25.0
8790,3/11/20,BR,Brazil,38.0,0.225806,7.0,6.0,31.0
8969,3/12/20,BR,Brazil,52.0,0.368421,14.0,7.0,38.0
9148,3/13/20,BR,Brazil,151.0,1.903846,99.0,14.0,52.0
9506,3/15/20,BR,Brazil,162.0,0.072848,11.0,99.0,151.0


In [53]:
X = target_country.loc[:, ['previous_day_total_cases']]
y = target_country['totalcases']

scaler = preprocessing.StandardScaler().fit(X)

clf = LogisticRegression(random_state=0).fit(scaler.transform(X), y)

In [59]:
clf.predict(scaler.transform([[162.0]]))

array([321.])